In [1]:
!pip install torch_geometric torch gdown --quiet

In [ ]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

In [2]:
%cd hackaton/

/home/onyxia/work/DL-Hackathon/hackaton


In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets


In [3]:
!ls -lh datasets

total 16K
drwxr-sr-x 2 onyxia users 4.0K May 24 07:10 A
drwxr-sr-x 2 onyxia users 4.0K May 24 07:11 B
drwxr-sr-x 2 onyxia users 4.0K May 24 07:11 C
drwxr-sr-x 2 onyxia users 4.0K May 24 07:11 D


In [4]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse

# Set the random seed
set_seed(42)


In [5]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [6]:
def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    return total_loss / len(data_loader),  correct / total

In [7]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
    return predictions

In [8]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [9]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [10]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [11]:
def get_arguments():
    args = {}
    args['train_path'] = get_user_input("Path to the training dataset (optional)")
    args['test_path'] = get_user_input("Path to the test dataset", required=True)
    args['num_checkpoints'] = get_user_input("Number of checkpoints to save during training", type_cast=int)
    args['device'] = get_user_input("Which GPU to use if any", default=1, type_cast=int)
    args['gnn'] = get_user_input("GNN type (gin, gin-virtual, gcn, gcn-virtual)", default='gin')
    args['drop_ratio'] = get_user_input("Dropout ratio", default=0.0, type_cast=float)
    args['num_layer'] = get_user_input("Number of GNN message passing layers", default=5, type_cast=int)
    args['emb_dim'] = get_user_input("Dimensionality of hidden units in GNNs", default=300, type_cast=int)
    args['batch_size'] = get_user_input("Input batch size for training", default=32, type_cast=int)
    args['epochs'] = get_user_input("Number of epochs to train", default=10, type_cast=int)
    args['baseline_mode'] = get_user_input("Baseline mode: 1 (CE), 2 (Noisy CE)", default=1, type_cast=int)
    args['noise_prob'] = get_user_input("Noise probability p (used if baseline_mode=2)", default=0.2, type_cast=float)

    
    return argparse.Namespace(**args)


In [12]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Path to the training dataset (optional) [None]:  datasets/B/train.json.gz
Path to the test dataset [None]:  datasets/B/test.json.gz
Number of checkpoints to save during training [None]:  30
Which GPU to use if any [1]:  1
GNN type (gin, gin-virtual, gcn, gcn-virtual) [gin]:  gcn
Dropout ratio [0.0]:  0.2
Number of GNN message passing layers [5]:  5
Dimensionality of hidden units in GNNs [300]:  300
Input batch size for training [32]:  8
Number of epochs to train [10]:  30
Baseline mode: 1 (CE), 2 (Noisy CE) [1]:  1
Noise probability p (used if baseline_mode=2) [0.2]:  0.2


Arguments received:
train_path: datasets/B/train.json.gz
test_path: datasets/B/test.json.gz
num_checkpoints: 30
device: 1
gnn: gcn
drop_ratio: 0.2
num_layer: 5
emb_dim: 300
batch_size: 8
epochs: 30
baseline_mode: 1
noise_prob: 0.2


In [13]:
class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (1 - torch.nn.functional.one_hot(targets, num_classes=logits.size(1)).float().sum(dim=1))
        return (losses * weights).mean()

In [14]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model1 = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
    model2 = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model1 = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
    model2 = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model1 = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
    model2 = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model1 = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
    model2 = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    criterion = NoisyCrossEntropyLoss(args.noise_prob)
else:
    criterion = torch.nn.CrossEntropyLoss(reduction='none')

In [15]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [16]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model1.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [17]:
def train_coteaching(train_loader, model1, model2, optimizer1, optimizer2, criterion, device, forget_rate, epoch, num_epochs):
    model1.train()
    model2.train()

    total_correct1 = total_correct2 = total_samples = 0
    running_loss1 = 0.0
    running_loss2 = 0.0

    for batch in train_loader:
        batch = batch.to(device, non_blocking=True)

        out1 = model1(batch)
        out2 = model2(batch)

        # Perte par échantillon (shape: [batch_size])
        loss1 = criterion(out1, batch.y)
        loss2 = criterion(out2, batch.y)

        remember_rate = 1.0 - forget_rate
        num_remember = int(remember_rate * batch.y.size(0))

        _, idx1 = loss1.topk(num_remember, largest=False)
        _, idx2 = loss2.topk(num_remember, largest=False)

        loss1_update = criterion(out1[idx2], batch.y[idx2]).mean()
        loss2_update = criterion(out2[idx1], batch.y[idx1]).mean()

        optimizer1.zero_grad()
        loss1_update.backward()
        optimizer1.step()

        optimizer2.zero_grad()
        loss2_update.backward()
        optimizer2.step()

        # On calcule la loss moyenne pondérée par la taille du batch (pour l’epoch)
        running_loss1 += loss1_update.item() * batch.y.size(0)
        running_loss2 += loss2_update.item() * batch.y.size(0)

        # Calcul des bonnes prédictions
        total_correct1 += (out1.argmax(dim=1) == batch.y).sum().item()
        total_correct2 += (out2.argmax(dim=1) == batch.y).sum().item()
        total_samples += batch.y.size(0)

    avg_loss1 = running_loss1 / total_samples
    avg_loss2 = running_loss2 / total_samples
    acc1 = total_correct1 / total_samples
    acc2 = total_correct2 / total_samples

    return acc1, acc2, avg_loss1, avg_loss2

In [18]:
if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0

    train_losses_model1 = []
    train_losses_model2 = []
    train_accuracies1 = []
    train_accuracies2 = []
    val_losses_model1 = []
    val_losses_model2 = []
    val_accuracies1 = []
    val_accuracies2 = []


    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    def get_forget_rate(epoch, total_epochs, max_forget=0.2):
        return min(max_forget, max_forget * epoch / (total_epochs * 0.5))

    for epoch in range(num_epochs):
        forget_rate = get_forget_rate(epoch, num_epochs)

        # Phase d'entraînement
        train_acc1, train_acc2, train_loss1, train_loss2 = train_coteaching(
            train_loader, model1, model2, optimizer1, optimizer2,
            criterion, device, forget_rate, epoch, num_epochs
        )

        # Phase de validation
        val_loss1, val_acc1 = evaluate(val_loader, model1, device, calculate_accuracy=True)
        val_loss2, val_acc2 = evaluate(val_loader, model2, device, calculate_accuracy=True)

        # Stocker les métriques
        train_losses_model1.append(train_loss1)
        train_losses_model2.append(train_loss2)
        train_accuracies1.append(train_acc1)
        train_accuracies2.append(train_acc2)
    
        val_losses_model1.append(val_loss1)
        val_losses_model2.append(val_loss2)
        val_accuracies1.append(val_acc1)
        val_accuracies2.append(val_acc2)
    
        print(f"Epoch {epoch+1}/{num_epochs} | Forget rate: {forget_rate:.3f}")
        print(f"Train Loss Model1: {train_loss1:.4f}, Acc: {train_acc1:.4f}")
        print(f"Train Loss Model2: {train_loss2:.4f}, Acc: {train_acc2:.4f}")
        print(f"Val   Loss Model1: {val_loss1:.4f}, Acc: {val_acc1:.4f}")
        print(f"Val   Loss Model2: {val_loss2:.4f}, Acc: {val_acc2:.4f}")
        
        if val_acc1 > best_val_accuracy:
            best_val_accuracy = val_acc1
            torch.save(model1.state_dict(), os.path.join(checkpoints_folder, f"model_{test_dir_name}"))
            print(f"[Co-Teaching] Best model1 updated and saved at {checkpoints_folder}/model_{test_dir_name}")

    # Assuming you have train_losses1 and train_losses2 collected similarly to train_accuracies
    plot_training_progress(train_losses_model1, train_accuracies1, os.path.join(logs_folder, "plots_model1"))
    plot_training_progress(val_losses_model1, val_accuracies1, os.path.join(logs_folder, "plotsVal_model1"))


Iterating eval graphs: 100%|██████████| 140/140 [00:14<00:00,  9.76batch/s]


Epoch 1/30 | Forget rate: 0.000
Train Loss Model1: 1.6972, Acc: 0.3482
Train Loss Model2: 1.7053, Acc: 0.3424
Val   Loss Model1: 1.6820, Acc: 0.4286
Val   Loss Model2: 1.7448, Acc: 0.3759
[Co-Teaching] Best model1 updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/B/model_B


Iterating eval graphs: 100%|██████████| 140/140 [00:25<00:00,  5.40batch/s]


Epoch 2/30 | Forget rate: 0.013
Train Loss Model1: 1.4731, Acc: 0.4071
Train Loss Model2: 1.4799, Acc: 0.4080
Val   Loss Model1: 1.6766, Acc: 0.4259
Val   Loss Model2: 1.9022, Acc: 0.4170


Iterating eval graphs: 100%|██████████| 140/140 [00:21<00:00,  6.51batch/s]


...

Epoch 28/30 | Forget rate: 0.200
Train Loss Model1: 0.8903, Acc: 0.5487
Train Loss Model2: 0.8936, Acc: 0.5491
Val   Loss Model1: 1.9278, Acc: 0.5384
Val   Loss Model2: 1.9599, Acc: 0.5188


Iterating eval graphs: 100%|██████████| 140/140 [00:11<00:00, 12.66batch/s]


Epoch 29/30 | Forget rate: 0.200
Train Loss Model1: 0.9113, Acc: 0.5469
Train Loss Model2: 0.9245, Acc: 0.5453
Val   Loss Model1: 1.7389, Acc: 0.5223
Val   Loss Model2: 1.8429, Acc: 0.5152


Iterating eval graphs: 100%|██████████| 140/140 [00:10<00:00, 12.94batch/s]


Epoch 30/30 | Forget rate: 0.200
Train Loss Model1: 0.9112, Acc: 0.5469
Train Loss Model2: 0.9083, Acc: 0.5424
Val   Loss Model1: 1.7818, Acc: 0.5402
Val   Loss Model2: 1.8276, Acc: 0.5429


In [ ]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

In [ ]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

In [ ]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)